# **Notebook to Manually Test LSTM Model**

In [1]:
import re
import numpy as np
import pickle
import torch
import torch.nn as nn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
  device = torch.device("cuda")
  print("GPU is available")
else:
  device = torch.device("cpu")
  print("GPU not available, CPU used")

GPU not available, CPU used


Define the model class

In [4]:
class LSTM(nn.Module):
  def __init__(self, no_layers, vocab_size, hidden_dim, embedding_dim=64, 
               output_dim=1, drop_prob=0.5, bidirectional=True):
    super(LSTM, self).__init__()

    self.output_dim = output_dim
    self.hidden_dim = hidden_dim
    self.no_layers = no_layers
    self.vocab_size = vocab_size

    # embedding and LSTM layers
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    
    #lstm
    self.lstm = nn.LSTM(input_size=embedding_dim,
                        hidden_size=self.hidden_dim,
                        num_layers=no_layers,
                        batch_first=True,
                        bidirectional=bidirectional)
    
    # dropout layer
    self.dropout = nn.Dropout(drop_prob)

    # linear and sigmoid layer
    self.fc = nn.Linear(self.hidden_dim, output_dim)
    self.sig = nn.Sigmoid()


  def forward(self, x, hidden):
    batch_size = x.size(0)

    # embeddings and lstm_out
    embeds = self.embedding(x)  # shape: B x S x Feature   since batch = True

    lstm_out, hidden = self.lstm(embeds, hidden)
    lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim) 
    
    # dropout and fully connected layer
    out = self.dropout(lstm_out)
    out = self.fc(out)
    sig_out = self.sig(out)
    
    # reshape to be batch_size first
    sig_out = sig_out.view(batch_size, -1)
    sig_out = sig_out[:, -1] # get last batch of labels
    
    # return last sigmoid output and hidden state
    return sig_out, hidden


  def init_hidden(self, batch_size):
    ''' Initializes hidden state '''

    # Create two new tensors with sizes (n_layers x 2) x batch_size x hidden_dim
    # initialized to zero, for hidden state and cell state of LSTM
    # n_layers x 2 because it is bidirectional LSTM
    h0 = torch.zeros((self.no_layers*2, batch_size, self.hidden_dim)).to(device)
    c0 = torch.zeros((self.no_layers*2, batch_size, self.hidden_dim)).to(device)
    hidden = (h0, c0)

    return hidden

Define functions to pre-process the raw text

In [5]:
def preprocess_string(s):
  '''
  Helper function to process a message
  '''
  # Remove all non-word characters (everything except numbers and letters)
  s = re.sub(r"[^\w\s]", '', s)

  # Replace all runs of whitespaces with no space
  s = re.sub(r"\s+", '', s)
  
  # replace digits with no space
  s = re.sub(r"\d", '', s)

  return s

def padding_(sentences, seq_len):
  features = np.zeros((len(sentences), seq_len), dtype=int)
  for ii, review in enumerate(sentences):
    if len(review) != 0:
      features[ii, -len(review):] = np.array(review)[:seq_len]
  return features

def predict(text, vocab, path='./drive/MyDrive/Colab Notebooks/Models/state_dict.pt'):
  word_seq = np.array([vocab[preprocess_string(word)] for word in text.lower().split() 
                         if preprocess_string(word) in vocab.keys()])
  word_seq = np.expand_dims(word_seq, axis=0)
  pad = torch.from_numpy(padding_(word_seq, 30))
  inputs = pad.to(device)
  batch_size = 1

  model = torch.load(path)
  model.eval()

  h = model.init_hidden(batch_size)
  h = tuple([each.data for each in h])

  output, h = model(inputs, h)
  return(output.item())

Load the model and the dictionary and get the input from the user.

In [8]:
a_file = open("./drive/MyDrive/Colab Notebooks/data.pkl", "rb")
vocab = pickle.load(a_file)

message = input("Enter message: ")
p = predict(message, vocab)
status = "bullish" if p > 0.5 else "bearish"
p = (1 - p) if status == "bearish" else p
print(f'Predicted status is {status} with a probability of {p}')

Enter message: Volume looking good
Predicted status is bullish with a probability of 0.994971752166748


# **Problematic cases**

"How can you be happy woth this price" - identified as bullish with high confidence but it is actually bearish, as the price is really low.

"It's not gonna rise" - negations not recognised due to stop-word removal

"Price not good" - again, classified as bullish due to lack of such cases in the training data.

"If it reaches 50, I'd be delighted" - bullish with low confidence. It implies bullish but the model classified as bullish almost by chance (51% bullish vs 49% bearish)

"I wish it was going up instead" - clearly bearish since the author wishes the opposite was happening to what is actually happening (dropping).